In [1]:
def display_images(images):
    grid = utils.make_grid(images)
    plt.imshow(grid.cpu().detach().numpy().transpose((1, 2, 0)))
    plt.show();

def format_data_for_display(tensor):
    maxVal = tensor.max()
    minVal = abs(tensor.min())
    maxVal = max(maxVal,minVal)
    output_data = tensor / maxVal
    output_data = output_data / 2
    output_data = output_data + 0.5
    return output_data

In [2]:
from __future__ import print_function
import zipfile
import os
import pdb
import torch
import h5py
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import datasets, transforms, utils

import numpy as np
import torch.nn as nn
import torch

output_height=32
output_width=32

class TransposeDepthInput(object):
    def __call__(self, depth):
        depth = depth.transpose((2, 0, 1))
        depth = torch.from_numpy(depth)
        depth = depth.view(1, depth.shape[0], depth.shape[1], depth.shape[2])
        depth = nn.functional.interpolate(depth, size=(output_height, output_width), mode='bilinear', align_corners=False)
        depth = torch.log(depth[0])
        return depth

rgb_data_transforms = transforms.Compose([
    transforms.Resize((output_height, output_width)),    # Different for Input Image & Depth Image
    transforms.ToTensor(),
])

depth_data_transforms = transforms.Compose([
    TransposeDepthInput(),
])

input_for_plot_transforms = transforms.Compose([
    transforms.Resize((output_height, output_width)),    # Different for Input Image & Depth Image
    transforms.ToTensor(),
])

class NYUDataset(Dataset):
    def __init__(self, filename, type, rgb_transform = None, depth_transform = None):
        f = h5py.File(filename, 'r')
        if type == "training":
            self.images = f['images'][0:1024]
            self.depths = f['depths'][0:1024]
        elif type == "validation":
            self.images = f['images'][1024:1248]
            self.depths = f['depths'][1024:1248]
        elif type == "test":
            self.images = f['images'][1248:]
            self.depths = f['depths'][1248:]
        self.rgb_transform = rgb_transform
        self.depth_transform = depth_transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        image = image.transpose((2, 1, 0))
        image = Image.fromarray(image)
        if self.rgb_transform:
            image = self.rgb_transform(image)
        depth = self.depths[idx]
        depth = np.reshape(depth, (1, depth.shape[0], depth.shape[1]))
        depth = depth.transpose((2, 1, 0))
        if self.depth_transform:
            depth = self.depth_transform(depth)
        sample = {'image': image, 'depth': depth}
        return sample

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNetConvBlock(nn.Module):
    def __init__(self, in_size, out_size, kernel_size=3, stride=2 ,activation=F.relu):
        super(UNetConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_size, out_size, kernel_size, padding=1, stride=2)
        self.activation = activation

    def forward(self, x):
        return self.activation(self.conv(x))

class UNetUpBlock(nn.Module):
    def __init__(self, in_size, out_size, kernel_size=3, stride=2, activation=F.relu, space_dropout=False):
        super(UNetUpBlock, self).__init__()
        self.up = nn.ConvTranspose2d(in_size, out_size, kernel_size=2, stride=2)
        self.activation = activation

    def forward(self, x):
        up = self.up(x)
        out = self.activation(up)
        return out

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        self.activation = F.relu

        self.conv_block3_16 = UNetConvBlock(3, 16)
        self.conv_block16_32 = UNetConvBlock(16, 32)

        self.up_block32_16 = UNetUpBlock(32, 16)
        self.up_block16_1 = UNetUpBlock(16, 1)

    def forward(self, x):
        block1 = self.conv_block3_16(x)
        block2 = self.conv_block16_32(block1)
        up1 = self.up_block32_16(block2)
        up2 = self.up_block16_1(up1)
        return up2


In [4]:
model = UNet()
from torchsummary import summary
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 16, 16]             448
     UNetConvBlock-2           [-1, 16, 16, 16]               0
            Conv2d-3             [-1, 32, 8, 8]           4,640
     UNetConvBlock-4             [-1, 32, 8, 8]               0
   ConvTranspose2d-5           [-1, 16, 16, 16]           2,064
       UNetUpBlock-6           [-1, 16, 16, 16]               0
   ConvTranspose2d-7            [-1, 1, 32, 32]              65
       UNetUpBlock-8            [-1, 1, 32, 32]               0
Total params: 7,217
Trainable params: 7,217
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.17
Params size (MB): 0.03
Estimated Total Size (MB): 0.21
----------------------------------------------------------------


In [8]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, utils
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

from torch.autograd import Variable
from logger import Logger
import pdb
import os
import re
import numpy as np

# Training settings
# parser = argparse.ArgumentParser(description='PyTorch depth map prediction example')
# parser.add_argument('model_folder', type=str, default='trial', metavar='F',
#                      help='In which folder do you want to save the model')
# parser.add_argument('--data', type=str, default='data', metavar='D',
#                      help="folder where data is located. train_data.zip and test_data.zip need to be found in the folder")
# parser.add_argument('--batch-size', type = int, default = 32, metavar = 'N',
#                      help='input batch size for training (default: 8)')
# parser.add_argument('--epochs', type=int, default = 10, metavar='N',
#                       help='number of epochs to train (default: 10)')
# parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
#                      help='learning rate (default: 0.01)')
# parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
#                      help='SGD momentum (default: 0.5)')
# parser.add_argument('--seed', type=int, default=1, metavar='S',
#                      help='random seed (default: 1)')
# parser.add_argument('--log-interval', type=int, default=10, metavar='N',
#                      help='how many batches to wait before logging training status')
# parser.add_argument('--suffix', type=str, default='', metavar='D',
#                      help='suffix for the filename of models and output files')
# args = parser.parse_args()

data = 'data'
batch_size = 1
epochs = 100
lr = 0.0001
momentum = 0.5
seed = 1
log_interval = 10
suffix = ''
model_folder = 'local-unet'

#torch.manual_seed(seed)

### Data Initialization and Loading
# from data import initialize_data, rgb_data_transforms, depth_data_transforms, output_height, output_width
#initialize_data(args.data) # extracts the zip files, makes a validation set

# from data import NYUDataset, rgb_data_transforms, depth_data_transforms, input_for_plot_transforms, output_height, output_width

# train_rgb_loader = torch.utils.data.DataLoader(datasets.ImageFolder(args.data + '/train_images/rgb/', transform = rgb_data_transforms), batch_size=args.batch_size, shuffle=False, num_workers=1)
# train_depth_loader = torch.utils.data.DataLoader(datasets.ImageFolder(args.data + '/train_images/depth/', transform = depth_data_transforms), batch_size=args.batch_size, shuffle=False, num_workers=1)
# val_rgb_loader = torch.utils.data.DataLoader(datasets.ImageFolder(args.data + '/val_images/rgb/', transform = rgb_data_transforms), batch_size=args.batch_size, shuffle=False, num_workers=1)
# val_depth_loader = torch.utils.data.DataLoader(datasets.ImageFolder(args.data + '/val_images/depth/', transform = depth_data_transforms), batch_size=args.batch_size, shuffle=False, num_workers=1)

train_loader = torch.utils.data.DataLoader(NYUDataset( 'nyu_depth_v2_labeled.mat', 
                                                       'training', 
                                                        rgb_transform = rgb_data_transforms, 
                                                        depth_transform = depth_data_transforms), 
                                                        batch_size = batch_size, 
                                                        shuffle = False, num_workers = 0)

val_loader = torch.utils.data.DataLoader(NYUDataset( 'nyu_depth_v2_labeled.mat',
                                                     'validation', 
                                                     rgb_transform = rgb_data_transforms, 
                                                     depth_transform = depth_data_transforms), 
                                                     batch_size = batch_size, 
                                                     shuffle = False, num_workers = 0)

model = UNet()
# model.cuda()

def rel_error(output, target):
    target = target + 0.000001
    target = log10(target)
    output = output + 0.000001
    output = log10(output)
    return F.mse_loss(output, target)

def custom_loss_function(output, target):
    di = target - output
    n = (output_height * output_width)
    di2 = torch.pow(di, 2)
    fisrt_term = torch.sum(di2,(1,2,3))/n
    second_term = 0.5*torch.pow(torch.sum(di,(1,2,3)), 2)/ (n**2)
    loss = fisrt_term - second_term
    return loss.sum()

#loss_function = custom_loss_function
loss_function = F.mse_loss
optimizer = optim.Adam(model.parameters(), amsgrad=True, lr=0.001)
dtype=torch.cuda.FloatTensor
logger = Logger('./logs/' + model_folder)

def plot_grid(fig, plot_input, output, actual_output, row_no):
        grid = ImageGrid(fig, 141, nrows_ncols=(row_no, 4), axes_pad=0.05, label_mode="1")
        for i in range(row_no):
                for j in range(3):
                        if(j == 0):
                                grid[i*4+j].imshow(np.transpose(plot_input[i], (1, 2, 0)), interpolation="nearest")
                        if(j == 1):
                                grid[i*4+j].imshow(np.transpose(output[i][0].detach().cpu().numpy(), (0, 1)), interpolation="nearest")
                        if(j == 2):
                                grid[i*4+j].imshow(np.transpose(actual_output[i][0].detach().cpu().numpy(), (0, 1)), interpolation="nearest")

def train_Unet(epoch):
    model.train()
    for batch_idx, data in enumerate(train_loader):
        rgb, depth = data['image'], data['depth']
        optimizer.zero_grad()
        output = model(rgb)
        target = depth[:,0,:,:].view(list(depth.shape)[0], 1, output_height, output_width)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
#             training_tag = "training loss epoch:" + str(epoch)
#             logger.scalar_summary(training_tag, loss.item(), batch_idx)

#             for tag, value in model.named_parameters():
#                 tag = tag.replace('.', '/') + ":" + str(epoch)
#                 logger.histo_summary(tag, value.data.cpu().numpy(), batch_idx)
#                 logger.histo_summary(tag + '/grad', value.grad.data.cpu().detach().numpy(), batch_idx)

            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(rgb), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        if epoch % 10 == 0:
            F = plt.figure(1, (30, 60))
            F.subplots_adjust(left=0.05, right=0.95)
            plot_grid(F, rgb, target, output, batch_size)
            plt.savefig("plots/train_" + model_folder + "_" + str(epoch) + "_" + str(batch_idx) + ".jpg")
            plt.show()
        if batch_idx == 0: break

def validate_Unet():
    print('validating unet')
    model.eval()
    validation_loss = 0
    with torch.no_grad():
        for batch_idx, data in enumerate(val_loader):
            rgb, depth = data['image'], data['depth']
            output = model(rgb)
            target = depth[:,0,:,:].view(list(depth.shape)[0], 1, output_height, output_width)
            validation_loss += rel_error(output, target)
            rel_loss = rel_error(output, target)
            rms_loss = F.mse_loss(output, target)
        validation_loss /= batch_idx
        rel_loss /= batch_idx
        rms_loss /= batch_idx
        logger.scalar_summary("validation loss", validation_loss, epoch)
        print('\nValidation set: Average loss: {:.6f} {:.6f} {:.6f}\n'.format(validation_loss, rel_loss, rms_loss))

folder_name = "models/" + model_folder
if not os.path.exists(folder_name): os.mkdir(folder_name)

for epoch in range(1, epochs + 1):
    print("********* Training the Unet Model **************")
    train_Unet(epoch)
    if epoch % 25== 0:
        model_file = folder_name + "/" + 'model_' + str(epoch) + '.pth'
        torch.save(model.state_dict(), model_file)
#    validate_Unet()

********* Training the Unet Model **************
Train Epoch: 1 [0/1024 (0%)]	Loss: 0.943549
********* Training the Unet Model **************
Train Epoch: 2 [0/1024 (0%)]	Loss: 0.933723
********* Training the Unet Model **************
Train Epoch: 3 [0/1024 (0%)]	Loss: 0.923531
********* Training the Unet Model **************
Train Epoch: 4 [0/1024 (0%)]	Loss: 0.912484
********* Training the Unet Model **************
Train Epoch: 5 [0/1024 (0%)]	Loss: 0.900400
********* Training the Unet Model **************
Train Epoch: 6 [0/1024 (0%)]	Loss: 0.886860
********* Training the Unet Model **************
Train Epoch: 7 [0/1024 (0%)]	Loss: 0.871501
********* Training the Unet Model **************
Train Epoch: 8 [0/1024 (0%)]	Loss: 0.854236
********* Training the Unet Model **************
Train Epoch: 9 [0/1024 (0%)]	Loss: 0.834649
********* Training the Unet Model **************
Train Epoch: 10 [0/1024 (0%)]	Loss: 0.812172
********* Training the Unet Model **************
Train Epoch: 11 [0/1

********* Training the Unet Model **************
Train Epoch: 91 [0/1024 (0%)]	Loss: 0.024196
********* Training the Unet Model **************
Train Epoch: 92 [0/1024 (0%)]	Loss: 0.023732
********* Training the Unet Model **************
Train Epoch: 93 [0/1024 (0%)]	Loss: 0.023298
********* Training the Unet Model **************
Train Epoch: 94 [0/1024 (0%)]	Loss: 0.022878
********* Training the Unet Model **************
Train Epoch: 95 [0/1024 (0%)]	Loss: 0.022463
********* Training the Unet Model **************
Train Epoch: 96 [0/1024 (0%)]	Loss: 0.022064
********* Training the Unet Model **************
Train Epoch: 97 [0/1024 (0%)]	Loss: 0.021687
********* Training the Unet Model **************
Train Epoch: 98 [0/1024 (0%)]	Loss: 0.021326
********* Training the Unet Model **************
Train Epoch: 99 [0/1024 (0%)]	Loss: 0.020978
********* Training the Unet Model **************
Train Epoch: 100 [0/1024 (0%)]	Loss: 0.020650


In [26]:
for batch_idx, data in enumerate(train_loader):
    rgb, depth = data['image'], data['depth']
    noisy_depth = depth + 0.1
    loss = custom_loss_function(depth, noisy_depth)
    print("depth:")
    print(depth)
    print("noisy depth:")
    print(noisy_depth)
    print("cust loss %.6f" % loss)
    print("mse loss %.6f" % F.mse_loss(depth, noisy_depth))
    print("l1 loss %.6f" % F.l1_loss(depth, noisy_depth))
    print("smooth l1 loss %.6f" % F.smooth_l1_loss(depth, noisy_depth))
    if batch_idx == 1: break

depth:
tensor([[[[1.0020, 1.0046, 1.0299,  ..., 0.8161, 0.7773, 0.7794],
          [0.9916, 0.9909, 0.9994,  ..., 0.7719, 0.7768, 0.7795],
          [0.9869, 0.9793, 0.9828,  ..., 0.7678, 0.7797, 0.7832],
          ...,
          [0.7032, 0.7053, 0.7219,  ..., 0.6852, 0.6584, 0.6599],
          [0.6525, 0.6451, 0.6386,  ..., 0.6874, 0.6606, 0.6618],
          [0.6252, 0.6051, 0.5937,  ..., 0.6794, 0.6641, 0.6635]]]])
noisy depth:
tensor([[[[1.1020, 1.1046, 1.1299,  ..., 0.9161, 0.8773, 0.8794],
          [1.0916, 1.0909, 1.0994,  ..., 0.8719, 0.8768, 0.8795],
          [1.0869, 1.0793, 1.0828,  ..., 0.8678, 0.8797, 0.8832],
          ...,
          [0.8032, 0.8053, 0.8219,  ..., 0.7852, 0.7584, 0.7599],
          [0.7525, 0.7451, 0.7386,  ..., 0.7874, 0.7606, 0.7618],
          [0.7252, 0.7051, 0.6937,  ..., 0.7794, 0.7641, 0.7635]]]])
cust loss 0.005000
mse loss 0.010000
l1 loss 0.099999
smooth l1 loss 0.005000
depth:
tensor([[[[0.6341, 0.6316, 0.6214,  ..., 0.3574, 0.3218, 0.3185],
 